In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
mobilenet_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/5"
train_dir     = "C:/Users/NAS/Desktop/TensorFlow/TL/TL-Feature Extraction/10_food_classes_10_percent/train/"
test_dir      = "C:/Users/NAS/Desktop/TensorFlow/TL/TL-Feature Extraction/10_food_classes_10_percent/test/"

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1/255.,
    rotation_range=0.2,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
    )

valid_datagen = ImageDataGenerator(
    rescale=1/255.,
    
)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=32,
    shuffle=True,
    class_mode="categorical"
)

valid_data = valid_datagen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=32,
    shuffle=False,
    class_mode="categorical"
)

In [ ]:
model = tf.keras.Sequential([
    hub.KerasLayer(mobilenet_url,trainable=False,input_shape=(224,224,3),name="mobilenet_layer"),
    tf.keras.layers.Dense(10,activation="softmax")
])

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

mobilenet_history = model.fit(
    train_data,
    epochs=5,
    steps_per_epoch=len(train_data),
    validation_data=valid_data,
    validation_steps=len(valid_data)
)

In [ ]:
epoch = range(len(mobilenet_history.history["loss"]))
plt.figure(figsize=(10,7))
plt.plot(epoch,mobilenet_history.history["val_loss"],label="val_loss")
plt.plot(epoch,mobilenet_history.history["loss"],label="loss")
plt.title("val_loss & loss")
plt.legend()
plt.figure(figsize=(10,7))
plt.plot(epoch,mobilenet_history.history["accuracy"],label="accuracy")
plt.plot(epoch,mobilenet_history.history["val_accuracy"],label="val_accuracy")
plt.title("accuracy & val_accuracy")
plt.legend()